### Housing price prediction 
This is second part of the California housing price prediction problem. In the first part I have explained how to perform end-to-end machine learning algorithm, and make prediction. 

This notebook is mainly about how to automate the preprocessing and test various models to find the best one for our case. 


#### Transformer 
Let's encapsulate custom cleanup, feature engineering in a custom class which will do the job seamlessly with sklearn functionalities like pipeline etc. 